In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/MURI Aug17 Thin Plate'

In [4]:
# !pip install smt

In [5]:
#Material Properties This link - https://www.mathworks.com/help/pde/ug/nonlinear-heat-transfer-in-a-thin-plate.html#heatTransferThinPlateExample-1
k = 400
rho = 8960
cp = 386
t_z = 0.01
stef_bolt = 5.670373e-8
hc = 1
Ta = 300
emiss = 0.5


In [6]:
label = "3D_HTTP_tanh"
loss_thresh = 20000
x = np.linspace(0,1,100).reshape(-1,1)
y = np.linspace(0,1,100).reshape(-1,1)
t = np.linspace(0,1,100).reshape(-1,1) #t is actually from 0 to 5000, let us scale it to 0 to 1

X,Y,T = np.meshgrid(x,y,t)

X = X.flatten('F').reshape(-1,1)
Y = Y.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xyt = np.hstack((X,Y,T))

initial_pts = np.logical_and(T==0,Y!=0).reshape(-1,)

DBC_pts = (Y == 0).reshape(-1,)


NBC_pts_x0 = (X == 0).reshape(-1,)
NBC_pts_x1 = (X == 1).reshape(-1,)

NBC_pts_y0 = (Y == 0).reshape(-1,)
NBC_pts_y1 = (Y == 1).reshape(-1,)

xyt_initial = xyt[initial_pts,:]
xyt_DBC = xyt[DBC_pts,:]

xyt_NBC_x0 = xyt[NBC_pts_x0,:]
xyt_NBC_x1 = xyt[NBC_pts_x1,:]

#xyt_NBC_y0 = xyt[NBC_pts_y0,:]
xyt_NBC_y1 = xyt[NBC_pts_y1,:]

u_initial = 300*np.ones((np.shape(xyt_initial)[0],1))
u_DBC = 1000*np.ones((np.shape(xyt_DBC)[0],1))

xyt_I_DBC = np.vstack((xyt_initial,xyt_DBC))
#xyt_NBC = np.vstack((xyt_NBC_1,xyt_NBC_2,xyt_NBC_3,xyt_NBC_4))
xyt_NBC_x = np.vstack((xyt_NBC_x0,xyt_NBC_x1))
#xyt_NBC_y = np.vstack((xyt_NBC_y0,xyt_NBC_y1))
xyt_NBC_y = np.vstack((xyt_NBC_y1))

u_I_DBC = np.vstack((u_initial,u_DBC))


lb_xyt = xyt[0]
ub_xyt = xyt[-1]

In [7]:
fea_data = scipy.io.loadmat('./../3D_HTTP_FEA.mat')
xy = fea_data['xy']
t = fea_data['t']/3000
xyt = np.zeros((497*101,3))
u_true = np.ones((497*101,1))


for i in range(101):
    t_temp = t[0,i]*np.ones((497,1))
    xyt[497*i:497*(i+1)] = np.hstack((xy,t_temp))
    u_true[497*i:497*(i+1)] = fea_data['u'][:,i].reshape(-1,1)
    #print(i)
#print(xyt)

xyt_test_tensor = torch.from_numpy(xyt).float().to(device)
u_true_norm = np.linalg.norm(u_true,2)

In [8]:
def trainingdata(N_D,N_N,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xyt_I_DBC.shape[0], N_D, replace=False) 
    xyt_D = xyt_I_DBC[idx,:] #choose indices from  set 'idx' (x,t)
    u_D = u_I_DBC[idx].reshape(-1,1)      #choose corresponding u

    idx = np.random.choice(xyt_NBC_x.shape[0], N_D, replace=False) 
    xyt_Nx = xyt_NBC_x[idx,:] #choose indices from  set 'idx' (x,t)

    idx = np.random.choice(xyt_NBC_y.shape[0], N_D, replace=False) 
    xyt_Ny = xyt_NBC_y[idx,:] #choose indices from  set 'idx' (x,t)

    '''Collocation Points'''
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    
    xyt_coll = lb_xyt + (ub_xyt - lb_xyt)*samples
    xyt_coll = np.vstack((xyt_coll, xyt_D,xyt_Nx,xyt_Ny)) # append training points to collocation points 

    return xyt_coll, xyt_D, u_D, xyt_Nx,xyt_Ny

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
    
            
    'foward pass'
    def forward(self,xyt):
        if torch.is_tensor(xyt) != True:         
            xyt = torch.from_numpy(xyt)                
        
        ubxyt = torch.from_numpy(ub_xyt).float().to(device)
        lbxyt = torch.from_numpy(lb_xyt).float().to(device)
    
                      
        #preprocessing input 
        xyt = (xyt - lbxyt)/(ubxyt - lbxyt)
        
        #convert to float
        a = xyt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z)         
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_D(self,xyt_D,u_D):
                
        loss_bc = self.loss_function(self.forward(xyt_D), u_D)
                
        return loss_bc
    
    def loss_N(self,xyt_Nx,xyt_Ny,N_hat):
        
        g1 = xyt_Nx.clone()             
        g1.requires_grad = True
        u1 = self.forward(g1)
        
        u1_x_y_t = autograd.grad(u1,g1,torch.ones([xyt_Nx.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du1_dx = u1_x_y_t[:,[0]]
        
        g2 = xyt_Ny.clone()             
        g2.requires_grad = True
        u2 = self.forward(g2)
        
        u2_x_y_t = autograd.grad(u2,g2,torch.ones([xyt_Ny.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        du2_dy = u2_x_y_t[:,[1]]
               
        loss_N1 = self.loss_function(du1_dx,N_hat)
        loss_N2 = self.loss_function(du2_dy,N_hat)
        
        #return loss_N1+loss_N2       
        return loss_N1 + loss_N2
    
    def loss_PDE(self, xyt_coll, f_hat):
        
        g = xyt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y_t = autograd.grad(u,g,torch.ones([xyt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy_tt = autograd.grad(u_x_y_t,g,torch.ones(xyt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dt = u_x_y_t[:,[2]]
        
        d2u_dx2 = u_xx_yy_tt[:,[0]]
        d2u_dy2 = u_xx_yy_tt[:,[1]]    
        

        f = rho*cp*t_z*du_dt/3000 - k*t_z*(d2u_dx2+d2u_dy2) + 2*hc*(u-Ta) + 2*emiss*stef_bolt*(torch.pow(u,4)-Ta**4) 
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat):

        loss_D = self.loss_D(xyt_D,u_D)
        loss_N = self.loss_N(xyt_Nx,xyt_Ny,N_hat)
        loss_f = self.loss_PDE(xyt_coll,f_hat)
        
        loss_val = loss_D + loss_N + loss_f
        
        #print(self.iter,"loss_D:",loss_D.cpu().detach().numpy(),"loss_N:",loss_N.cpu().detach().numpy(),"loss_f:",loss_f.cpu().detach().numpy())
        
        return loss_val
       
    'test neural network'
    def test(self):
        u_pred = self.forward(xyt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
        
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xyt_coll_np_array, xyt_D_np_array, u_D_np_array,xyt_Nx_np_array,xyt_Ny_np_array = trainingdata(N_D,N_N,N_f,(reps)*22)

    xyt_coll = torch.from_numpy(xyt_coll_np_array).float().to(device)
    xyt_D = torch.from_numpy(xyt_D_np_array).float().to(device)
    u_D = torch.from_numpy(u_D_np_array).float().to(device)
    xyt_Nx = torch.from_numpy(xyt_Nx_np_array).float().to(device)
    xyt_Ny = torch.from_numpy(xyt_Ny_np_array).float().to(device)

    N_hat = torch.zeros(xyt_Nx.shape[0],1).to(device)    
    f_hat = torch.zeros(xyt_coll.shape[0],1).to(device)

    
    for i in range(max_iter):
        train_step(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat,i)

        loss_np = PINN.loss(xyt_D,u_D,xyt_Nx,xyt_Ny,N_hat,xyt_coll,f_hat).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
max_reps = 10
max_iter = 100

train_loss_full = []
test_mse_full = []
test_re_full = []

elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):
    print(label)
    train_loss = []
    test_mse_loss = []
    test_re_loss = []

    print(reps)

    torch.manual_seed(reps*36)
    N_D = 5000 #Total number of data points for 'y'
    N_N = 3500
    N_f = 10000 #Total number of collocation points 

    layers = np.array([3,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())


    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



    nan_flag = train_model(max_iter,reps)


    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time

    #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

3D_HTTP_tanh
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 669958.2 Test MSE 301439.9805083001 Test RE 0.9732844588219348
1 Train Loss 427549.66 Test MSE 189295.05827936932 Test RE 0.7712745489175916
2 Train Loss 237088.25 Test MSE 80406.88213272895 Test RE 0.502673479919789
3 Train Loss 222040.33 Test MSE 69917.21104648277 Test 

96 Train Loss 71687.99 Test MSE 14127.331900407955 Test RE 0.2107023776516001
97 Train Loss 71566.51 Test MSE 14149.395550615476 Test RE 0.21086684783934262
98 Train Loss 71236.41 Test MSE 14132.023213811557 Test RE 0.21073735909210764
99 Train Loss 70944.484 Test MSE 14048.688090164482 Test RE 0.21011509183753424
Training time: 520.70
3D_HTTP_tanh
1
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linea

91 Train Loss 37063.54 Test MSE 4615.467181767097 Test RE 0.12043346893382474
92 Train Loss 36642.457 Test MSE 4724.57817649688 Test RE 0.12184869492435513
93 Train Loss 36332.81 Test MSE 4607.945952686108 Test RE 0.12033530151181346
94 Train Loss 36124.266 Test MSE 4545.066530332033 Test RE 0.1195114414644727
95 Train Loss 35965.902 Test MSE 4372.975035377841 Test RE 0.11722705676095856
96 Train Loss 35802.52 Test MSE 4379.685959231397 Test RE 0.11731697269362606
97 Train Loss 35622.17 Test MSE 4443.202425646244 Test RE 0.11816460619757904
98 Train Loss 35445.56 Test MSE 4393.284257072865 Test RE 0.1174989577453938
99 Train Loss 35068.367 Test MSE 4187.083980729814 Test RE 0.1147083936472094
Training time: 571.02
3D_HTTP_tanh
2
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=3, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50,

87 Train Loss 192421.03 Test MSE 53592.23146623412 Test RE 0.4103838698303741
88 Train Loss 191951.2 Test MSE 52928.52748219896 Test RE 0.4078347880403371
89 Train Loss 191684.19 Test MSE 52980.298582043615 Test RE 0.4080341974701358
90 Train Loss 191406.05 Test MSE 53016.94353707649 Test RE 0.4081752858612573
91 Train Loss 191325.97 Test MSE 52997.25735399516 Test RE 0.40809949726487704
92 Train Loss 191179.55 Test MSE 52910.689846730966 Test RE 0.4077660593069203
93 Train Loss 190914.62 Test MSE 52555.98154974799 Test RE 0.4063969481617851
94 Train Loss 190379.45 Test MSE 51959.28082311921 Test RE 0.4040833238299138
95 Train Loss 189827.7 Test MSE 51706.85531898904 Test RE 0.4031005819532103
96 Train Loss 189678.66 Test MSE 51850.87512552741 Test RE 0.4036615723547375
97 Train Loss 189645.58 Test MSE 51842.675674607715 Test RE 0.40362965453172917
98 Train Loss 189572.19 Test MSE 51754.76139485203 Test RE 0.403287273796476
99 Train Loss 189488.45 Test MSE 51738.222159073106 Test RE 0.

82 Train Loss 110239.29 Test MSE 25064.29708581044 Test RE 0.2806513444225859
83 Train Loss 109743.03 Test MSE 24588.137946840277 Test RE 0.2779727238172826
84 Train Loss 109502.15 Test MSE 24600.954433995998 Test RE 0.27804516056951395
85 Train Loss 109397.15 Test MSE 24631.82155435918 Test RE 0.2782195392307326
86 Train Loss 109008.05 Test MSE 24328.235169609252 Test RE 0.2764997002588802
87 Train Loss 108818.77 Test MSE 24408.0681064565 Test RE 0.27695299462783823
88 Train Loss 108594.07 Test MSE 24430.55814813475 Test RE 0.27708056003407133
89 Train Loss 108530.09 Test MSE 24321.907540747823 Test RE 0.2764637399590383
90 Train Loss 108324.14 Test MSE 24263.412747176877 Test RE 0.27613108873892633
91 Train Loss 108021.02 Test MSE 24202.54654756538 Test RE 0.27578452571105966
92 Train Loss 107786.69 Test MSE 24005.409729773794 Test RE 0.27465905640906235
93 Train Loss 107618.0 Test MSE 23872.787026750237 Test RE 0.27389930106511695
94 Train Loss 107499.164 Test MSE 23641.32760074703 

...

72 Train Loss 128383.86 Test MSE 30714.07018547214 Test RE 0.3106762457019279
73 Train Loss 127584.02 Test MSE 30323.88350620295 Test RE 0.3086965472709711
74 Train Loss 127156.79 Test MSE 30630.224356421837 Test RE 0.31025190094468325
75 Train Loss 126446.78 Test MSE 30567.47440949841 Test RE 0.30993394259185036
76 Train Loss 125442.39 Test MSE 29898.792619225114 Test RE 0.3065252022420726
77 Train Loss 125015.91 Test MSE 29684.362815073375 Test RE 0.305424047212306
78 Train Loss 123951.69 Test MSE 29198.048094983376 Test RE 0.3029118560067036
79 Train Loss 123249.09 Test MSE 29007.32858776288 Test RE 0.3019209361958588
80 Train Loss 122788.164 Test MSE 28631.541288444412 Test RE 0.2999588818099443
81 Train Loss 121254.01 Test MSE 28334.566825128622 Test RE 0.298399197615804
82 Train Loss 120739.38 Test MSE 28128.21925660605 Test RE 0.29731066037107806
83 Train Loss 120202.25 Test MSE 28030.438340944682 Test RE 0.2967934463735472
84 Train Loss 119377.6 Test MSE 27679.59630278514 Test 

68 Train Loss 139300.94 Test MSE 33593.16113035229 Test RE 0.3249112872184811
69 Train Loss 138418.69 Test MSE 33090.04657153974 Test RE 0.32246905982548124
70 Train Loss 137759.88 Test MSE 32890.36321311474 Test RE 0.321494610566461
71 Train Loss 137213.7 Test MSE 32777.656652336686 Test RE 0.3209432994349492
72 Train Loss 136625.27 Test MSE 32501.222600566598 Test RE 0.3195870782236806
73 Train Loss 135939.73 Test MSE 32676.548086117917 Test RE 0.3204479135016059
74 Train Loss 135412.94 Test MSE 32554.35839926349 Test RE 0.31984821577872435
75 Train Loss 134879.53 Test MSE 32388.645479550818 Test RE 0.31903310825701625
76 Train Loss 134128.17 Test MSE 32040.06572338141 Test RE 0.31731168211810223
77 Train Loss 133671.66 Test MSE 31860.008496255497 Test RE 0.3164188194356086
78 Train Loss 133222.5 Test MSE 31773.196546734926 Test RE 0.3159874373800358
79 Train Loss 133016.55 Test MSE 31813.868182497317 Test RE 0.316189614328474
80 Train Loss 132510.38 Test MSE 31626.557098748264 Test 

64 Train Loss 174267.44 Test MSE 44360.21255086381 Test RE 0.3733672073267263
65 Train Loss 174152.86 Test MSE 44189.23836557784 Test RE 0.37264699230814624
66 Train Loss 173922.81 Test MSE 44038.61850165956 Test RE 0.37201136300779425
67 Train Loss 173815.47 Test MSE 43804.177413771344 Test RE 0.371019834046575
68 Train Loss 173289.36 Test MSE 43665.50327678102 Test RE 0.3704320859896004
69 Train Loss 173022.2 Test MSE 43633.645525822416 Test RE 0.3702969302541974
70 Train Loss 172937.48 Test MSE 43530.5729717848 Test RE 0.3698593090493244
71 Train Loss 172621.67 Test MSE 43402.24397138027 Test RE 0.3693137303353113
72 Train Loss 172114.12 Test MSE 43031.352982996585 Test RE 0.3677323717819676
73 Train Loss 171874.83 Test MSE 43052.729591820826 Test RE 0.36782369931995335
74 Train Loss 171783.14 Test MSE 43013.34470029016 Test RE 0.3676554171775352
75 Train Loss 171619.25 Test MSE 42781.255703160576 Test RE 0.3666621883051776
76 Train Loss 171387.19 Test MSE 42754.17143757578 Test RE 

60 Train Loss 146948.5 Test MSE 33955.13516439677 Test RE 0.32665709385978553
61 Train Loss 146624.77 Test MSE 33837.57571218926 Test RE 0.32609112754840097
62 Train Loss 146438.12 Test MSE 33955.184356931306 Test RE 0.32665733048208845
63 Train Loss 146245.45 Test MSE 33895.76233803234 Test RE 0.32637137801135396
64 Train Loss 146016.16 Test MSE 33918.74117552899 Test RE 0.3264819871807887
65 Train Loss 145617.58 Test MSE 33781.52750540473 Test RE 0.32582094865375766
66 Train Loss 145249.5 Test MSE 33651.67687356894 Test RE 0.32519414469013286
67 Train Loss 144785.81 Test MSE 33338.3757636363 Test RE 0.3236768067203347
68 Train Loss 144438.2 Test MSE 33222.13187440607 Test RE 0.32311201756499786
69 Train Loss 143927.86 Test MSE 32980.96397869982 Test RE 0.3219371053218955
70 Train Loss 143571.95 Test MSE 32979.14601614944 Test RE 0.32192823236010304
71 Train Loss 142604.98 Test MSE 32521.021777236074 Test RE 0.3196844068339656
72 Train Loss 142125.6 Test MSE 32333.49191740533 Test RE 

56 Train Loss 181829.92 Test MSE 52557.24725632875 Test RE 0.40640184176456456
57 Train Loss 181637.17 Test MSE 52293.333796483195 Test RE 0.40538019484995824
58 Train Loss 181135.88 Test MSE 52011.78534900847 Test RE 0.404287434106247
59 Train Loss 180523.97 Test MSE 51511.86434381974 Test RE 0.4023398006420476
60 Train Loss 180357.22 Test MSE 51594.403829629075 Test RE 0.402662014061675
61 Train Loss 179866.23 Test MSE 51366.60560647857 Test RE 0.4017721194718857
62 Train Loss 179583.89 Test MSE 51237.37469543834 Test RE 0.40126640107547096
63 Train Loss 179303.7 Test MSE 50704.55970399272 Test RE 0.39917457367222403
64 Train Loss 178533.66 Test MSE 50477.97108953492 Test RE 0.3982816590159997
65 Train Loss 178256.34 Test MSE 50274.64029780333 Test RE 0.3974786885320812
66 Train Loss 177761.95 Test MSE 50162.657399719756 Test RE 0.3970357651322036
67 Train Loss 177448.39 Test MSE 50142.0153550912 Test RE 0.3969540661781274
68 Train Loss 177225.17 Test MSE 49877.23686234578 Test RE 0.

In [14]:
import scipy.io as sio

In [15]:
for tune_reps in range(5):
    label = "3D_HTTP_tanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: '3D_HTTP_tanh_tune0.mat'